# 样例介绍

功能：对蛋白质图像进行自动化分类评估  
样例输入：未标注的蛋白质荧光显微图片  
样例输出：已经标注分类的蛋白质图谱

# 前期准备

基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考如下步骤：
1. 获取模型和测试数据。我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，将模型和测试数据下载到本地，并放到本样例目录下。
2. 模型转换。利用atc工具将原始模型转换为om模型，转换命令如下：  
    ```shell
    atc --model=./hpa.prototxt --weight=./hpa.caffemodel --framework=0 --output=./deploy_vel  --soc_version=Ascend310B1 --input_format=NCHW --input_fp16_nodes=data --output_type=FP32 --out_nodes="score:0" 
    ```

    其中各个参数具体含义如下：
    * --model：原始模型文件。
    * --weight：原始模型权重。
    * --framework：原始框架类型,  0: Caffe, 1: MindSpore, 3: TensorFlow, 5: ONNX。
    * --output：输出的模型文件路径。
    * --soc_version：昇腾AI处理器型号，此处为"Ascend310B1"。
    * --input_format：输入Tensor的内存排列方式。
    * --input_fp16_nodes：指定输入数据类型为FP16的输入节点名称。
    * --output_type：此处为指定某个输出节点的输出类型，需要和--out_nodes参数配合使用。
    * --out_nodes：指定某个输出节点。

# 模型推理实现

### 1. 导入三方库

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

import acl
import acllite_utils as utils
import constants as constants
from acllite_model import AclLiteModel
from acllite_resource import resource_list

### 2. 定义acllite资源初始化与去初始化类

In [ ]:
class AclLiteResource:
    """
    AclLiteResource
    """
    def __init__(self, device_id=0):
        self.device_id = device_id
        self.context = None
        self.stream = None
        self.run_mode = None
        
    def init(self):
        """
        init resource
        """
        print("init resource stage:")
        ret = acl.init()

        ret = acl.rt.set_device(self.device_id)
        utils.check_ret("acl.rt.set_device", ret)

        self.context, ret = acl.rt.create_context(self.device_id)
        utils.check_ret("acl.rt.create_context", ret)

        self.stream, ret = acl.rt.create_stream()
        utils.check_ret("acl.rt.create_stream", ret)

        self.run_mode, ret = acl.rt.get_run_mode()
        utils.check_ret("acl.rt.get_run_mode", ret)

        print("Init resource success")

    def __del__(self):
        print("acl resource release all resource")
        resource_list.destroy()
        if self.stream:
            print("acl resource release stream")
            acl.rt.destroy_stream(self.stream)

        if self.context:
            print("acl resource release context")
            acl.rt.destroy_context(self.context)

        print("Reset acl device ", self.device_id)
        acl.rt.reset_device(self.device_id)
        print("Release acl resource success")

### 3. 定义模型类，包含前处理、推理、后处理等操作

In [ ]:
class Hpa(object):
    """
    Class for portrait segmentation
    """
    def __init__(self, model_path, model_width, model_height):
        self._model_path = model_path
        self._model_width = model_width
        self._model_height = model_height
        self._img_width = 0
        self._img_height = 0
        self._model = None

    def init(self):
        """
        Initialize
        """
        # Load model
        self._model = AclLiteModel(self._model_path)

        return constants.SUCCESS

    def pre_process(self, im):
        """
        image preprocess
        """
        # 图片缩放
        self._img_width = im.size[0]
        self._img_height = im.size[1]
        im = im.resize((224, 224))
        img = np.array(im)
        # rgb to bgr, HWC to CHW
        img = img[:, :, ::-1]
        img = img.astype("float16")
        result = img.transpose([2, 0, 1]).copy() 
        return result 

    def inference(self, input_data):
        """
        model inference
        """
        return self._model.execute(input_data)  # 推理

    def sigmoid(self, x):
        """
        sigmod function
        """
        return 1. / (1 + np.exp(-x))  # 对矩阵每个元素执行 sigmoid 操作

    def visualize(self, file_name, pred):    
        """
        visualize
        """

        # 类别名称
        id_2_label = [
            "Mitochondria", "Nucleus", "Endoplasmic reticulum", "Nuclear speckles", 
            "Plasma membrane", "Nucleoplasm", "Cytosol", "Nucleoli",
            "Vesicles", "Golgi apparatus"
        ]
 
        # 读取图片
        setFont = ImageFont.truetype('font.ttf', 20)
        fillColor = "#fff"
        im = Image.open(file_name)
        im = im.resize((512, 512))
        draw = ImageDraw.Draw(im)
        pred = pred.flatten() 
        
        # 在图片上画出top1的类别名称与置信度
        top1 = np.argmax(pred)
        print(id_2_label[top1], pred[top1])
        label =  "%s : %.2f%%" % (id_2_label[top1], float(pred[top1]) * 100)
        pred[top1] = 0
        draw.text(xy = (20, 20), text = label, font=setFont, fill=fillColor)
        
        # 在图片上画出top2的类别名称与置信度
        top2 = np.argmax(pred)
        print(top2, pred.shape)
        label =  "%s : %.2f%%" % (id_2_label[top2], float(pred[top2]) * 100)
        pred[top2] = 0
        draw.text(xy = (20, 50), text = label, font=setFont, fill=fillColor)
        
        # 在图片上画出top3的类别名称与置信度
        top3 = np.argmax(pred)
        label =  "%s : %.2f%%" % (id_2_label[top3], float(pred[top3]) * 100)
        pred[top3] = 0
        draw.text(xy = (20, 80), text = label, font=setFont, fill=fillColor)
 
        # show photo
        plt.axis('off')
        plt.xticks([])
        plt.yticks([])
        plt.imshow(im)


    def post_process(self, result, image_name):  
        """
        post_process
        """   
        # 将模型输出概率压缩到0-1范围内
        score = np.array(result[0])
        pred = self.sigmoid(score)

        # visualize
        self.visualize(image_name, pred)

### 4. 构造主函数，串联整个代码逻辑

In [ ]:
def main():
    MODEL_PATH = 'deploy_vel.om'
    MODEL_WIDTH = 224
    MODEL_HEIGHT = 224
    
    # 初始化acl资源
    acl_resource = AclLiteResource()
    acl_resource.init()
    
    # 初始化模型
    hpa = Hpa(MODEL_PATH, MODEL_WIDTH, MODEL_HEIGHT)
    ret = hpa.init()
    utils.check_ret("hpa init ", ret)
    
    # 读取图片
    image_name = 'test.jpeg'
    print('====' + image_name + '====')
    im = Image.open(image_name)
    if len(im.split()) != 3:
        print('warning: "{}" is not a color image and will be ignored'.format(image_name))

    # 前处理
    resized_image = hpa.pre_process(im)

    # 推理
    result = hpa.inference([resized_image, ])

    # 后处理
    hpa.post_process(result, image_name)

### 5. 运行
运行完成后，会显示推理后的图片，推理结果中的置信度排名前三的类别名称，会显示在图片左上角，如下所示。推理的top1结果为Mitochondria，相应地，其置信度为64.23%，top2和top3分别为nucleoli和Nucleoplasm，相应地，其置信度为40.25%和0.41%。

In [ ]:
main()

# 样例总结

我们来回顾一下以上代码，可以包括以下几个步骤：
1. 初始化acl资源：在调用acl相关资源时，必须先初始化AscendCL，否则可能会导致后续系统内部资源初始化出错。  
2. 对图片进行前处理：在此样例中，我们首先利用PIL读入图片，再对图片进行缩放、格式转换、维度转换等。
3. 推理：利用AclLiteModel.execute接口对图片进行推理。    
4. 对推理结果进行后处理：把模型输出利用sigmoid压缩到0-1范围内，转化为相应概率值，也即置信度。  
5. 可视化图片：提取出置信度前三的分类结果，将其画在原图上。